#  Speech Recognition-Based Sentiment Analysis and Quiz Generator

## 1. Conversion of speech to text and seperating using commas

In [5]:
import speech_recognition as sr
import csv
from textblob import TextBlob

def capture_speech():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, 1)  # Adjust for ambient noise
        print("Say something!")
        audio = r.listen(source)
    try:
        recognized_text = r.recognize_google(audio, language='en-IN')
        print("Recognized text: " + recognized_text)
        return recognized_text
    except Exception as e:
        print("Error: " + str(e))
        return ""

def separate_words_with_commas(text):
    words = text.split()
    comma_separated_text = ','.join(words)
    return words, comma_separated_text

def save_to_csv(words, filename='words.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Word"])
        for word in words:
            writer.writerow([word])

def classify_words(words):
    pros = []
    cons = []
    for i, word in enumerate(words):
        context = " ".join(words[max(0, i-2):min(len(words), i+3)])
        analysis = TextBlob(context)
        if analysis.sentiment.polarity > 0:
            pros.append(word)
        else:
            cons.append(word)
    return pros, cons

if __name__ == "__main__":
    text = capture_speech()
    if text:
        words, comma_separated_text = separate_words_with_commas(text)
        save_to_csv(words)
        pros, cons = classify_words(words)
        print("Pros:", pros)
        print("Cons:", cons)


Say something!
Recognized text: I am good in Java you are bad in Python I know very good Java but don't know python
Pros: ['I', 'am', 'good', 'in', 'Java', 'I', 'know', 'very', 'good', 'Java', 'but']
Cons: ['you', 'are', 'bad', 'in', 'Python', "don't", 'know', 'python']


## 2. Incorporating customizable questions 

In [5]:
import speech_recognition as sr
import csv
from textblob import TextBlob
import re

def capture_speech():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, 1)  # Adjust for ambient noise
        print("Say something!")
        audio = r.listen(source)
    try:
        recognized_text = r.recognize_google(audio, language='en-IN')
        print("Recognized text: " + recognized_text)
        return recognized_text
    except Exception as e:
        print("Error: " + str(e))
        return ""

def separate_words_with_commas(text):
    words = text.split()
    comma_separated_text = ','.join(words)
    return words, comma_separated_text

def save_to_csv(words, filename='words.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Word"])
        for word in words:
            writer.writerow([word])

# def classify_words(words):
#     pros = []
#     cons = []
#     for i, word in enumerate(words):
#         context = " ".join(words[max(0, i-2):min(len(words), i+3)])
#         analysis = TextBlob(context)
#         if analysis.sentiment.polarity > 0:
#             pros.append(word)
#         else:
#             cons.append(word)
#     return pros, cons

def classify_words(words):
    pros = []
    cons = []
    window_size = 5  # Increase the context window size

    for i, word in enumerate(words):
        start = max(0, i - window_size)
        end = min(len(words), i + window_size + 1)
        context = " ".join(words[start:end])
        analysis = TextBlob(context)
        
        if analysis.sentiment.polarity > 0:
            pros.append(word)
        elif analysis.sentiment.polarity < 0:
            cons.append(word)
        else:
            # If neutral, check for positive words in context
            positive_words = ['good', 'great', 'excellent', 'awesome', 'like', 'love']
            if any(pos_word in context.lower() for pos_word in positive_words):
                pros.append(word)
            else:
                cons.append(word)

    return pros, cons
    
def generate_questions(pros):
    questions = []
    for word in pros:
        if word.lower() in ['python', 'java', 'javascript', 'cpp','c']:
            questions.append({
                "question": f"What are the main advantages of {word}?",
                "answer": f"The main advantages of {word} include its ease of use, versatility, and large community support. It's widely used in various domains and has extensive libraries and frameworks."
            })
        elif word.lower() in ['function', 'method', 'class']:
            questions.append({
                "question": f"What is the purpose of a {word} in programming?",
                "answer": f"A {word} in programming is used to organize and structure code. It helps in code reusability, modularity, and makes the code easier to understand and maintain."
            })
        elif word.lower() in ['algorithm', 'data structure']:
            questions.append({
                "question": f"Why is understanding {word}s important in programming?",
                "answer": f"Understanding {word}s is crucial in programming as they help in solving complex problems efficiently. They are fundamental to writing optimized and scalable code."
            })
       
    return questions

def ask_questions(questions):
    scores = []
    detailed_grades = []
    for i, q in enumerate(questions, 1):
        print(f"{i}. {q['question']}")
        answer = input("Your answer: ")
        
        # Detailed grading
        keyword_score = sum(2 for keyword in q["answer"].lower().split() if keyword in answer.lower())
        length_score = min(len(answer.split()) / 5, 4)  # Up to 4 points for length
        
        question_score = min(keyword_score + length_score, 10)  # Max 10 points per question
        scores.append(question_score)
        
        detailed_grade = {
            "question": q["question"],
            "user_answer": answer,
            "correct_answer": q["answer"],
            "keyword_score": keyword_score,
            "length_score": length_score,
            "total_score": question_score
        }
        detailed_grades.append(detailed_grade)
    
    return scores, detailed_grades

def display_results(scores, detailed_grades):
    print("\nDetailed Grading Results:")
    total_score = sum(scores)
    print(f"Total Score: {total_score:.2f}/{len(scores)*10}")
    for i, grade in enumerate(detailed_grades, 1):
        print(f"\nQuestion {i}:")
        print(f"  Q: {grade['question']}")
        print(f"  Your Answer: {grade['user_answer']}")
        print(f"  Correct Answer: {grade['correct_answer']}")
        print(f"  Keyword Score: {grade['keyword_score']}/10")
        print(f"  Length Score: {grade['length_score']:.2f}/4")
        print(f"  Total Question Score: {grade['total_score']:.2f}/10")

if __name__ == "__main__":
    text = capture_speech()
    if text:
        words, comma_separated_text = separate_words_with_commas(text)
        save_to_csv(words)
        pros, cons = classify_words(words)
        print("Pros:", pros)
        print("Cons:", cons)
        
        questions = generate_questions(pros)
        if questions:
            print("\nQuestions based on the pros:")
            scores, detailed_grades = ask_questions(questions)
            display_results(scores, detailed_grades)
        else:
            print("\nNo questions generated from the pros list.")

Say something!
Recognized text: I am very good in Java and Python but I know only Basics of C
Pros: ['I', 'am', 'very', 'good', 'in', 'Java', 'and', 'Python', 'but']
Cons: ['I', 'know', 'only', 'Basics', 'of', 'C']

Questions based on the pros:
1. What are the main advantages of Java?


Your answer:  Java is a programming language.


2. What are the main advantages of Python?


Your answer:  Python is a reptile.



Detailed Grading Results:
Total Score: 7.80/20

Question 1:
  Q: What are the main advantages of Java?
  Your Answer: Java is a programming language.
  Correct Answer: The main advantages of Java include its ease of use, versatility, and large community support. It's widely used in various domains and has extensive libraries and frameworks.
  Keyword Score: 4/10
  Length Score: 1.00/4
  Total Question Score: 5.00/10

Question 2:
  Q: What are the main advantages of Python?
  Your Answer: Python is a reptile.
  Correct Answer: The main advantages of Python include its ease of use, versatility, and large community support. It's widely used in various domains and has extensive libraries and frameworks.
  Keyword Score: 2/10
  Length Score: 0.80/4
  Total Question Score: 2.80/10


## 3. Generating a report

In [6]:
import speech_recognition as sr
import csv
from textblob import TextBlob
import re

def capture_speech():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, 1)  # Adjust for ambient noise
        print("Say something!")
        audio = r.listen(source)
    try:
        recognized_text = r.recognize_google(audio, language='en-IN')
        print("Recognized text: " + recognized_text)
        return recognized_text
    except Exception as e:
        print("Error: " + str(e))
        return ""

def separate_words_with_commas(text):
    words = text.split()
    comma_separated_text = ','.join(words)
    return words, comma_separated_text

def save_to_csv(words, filename='words.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Word"])
        for word in words:
            writer.writerow([word])

def classify_words(words):
    pros = []
    cons = []
    window_size = 5  # Increase the context window size

    for i, word in enumerate(words):
        start = max(0, i - window_size)
        end = min(len(words), i + window_size + 1)
        context = " ".join(words[start:end])
        analysis = TextBlob(context)
        
        if analysis.sentiment.polarity > 0:
            pros.append(word)
        elif analysis.sentiment.polarity < 0:
            cons.append(word)
        else:
            # If neutral, check for positive words in context
            positive_words = ['good', 'great', 'excellent', 'awesome', 'like', 'love']
            if any(pos_word in context.lower() for pos_word in positive_words):
                pros.append(word)
            else:
                cons.append(word)

    return pros, cons
    
def generate_questions(pros):
    questions = []
    for word in pros:
        if word.lower() in ['python', 'java', 'javascript', 'cpp', 'c']:
            questions.append({
                "question": f"What are the main advantages of {word}?",
                "answer": f"The main advantages of {word} include its ease of use, versatility, and large community support. It's widely used in various domains and has extensive libraries and frameworks."
            })
        elif word.lower() in ['function', 'method', 'class']:
            questions.append({
                "question": f"What is the purpose of a {word} in programming?",
                "answer": f"A {word} in programming is used to organize and structure code. It helps in code reusability, modularity, and makes the code easier to understand and maintain."
            })
        elif word.lower() in ['algorithm', 'data structure']:
            questions.append({
                "question": f"Why is understanding {word}s important in programming?",
                "answer": f"Understanding {word}s is crucial in programming as they help in solving complex problems efficiently. They are fundamental to writing optimized and scalable code."
            })
       
    return questions

def ask_questions(questions):
    scores = []
    detailed_grades = []
    for i, q in enumerate(questions, 1):
        print(f"{i}. {q['question']}")
        answer = input("Your answer: ")
        
        # Detailed grading
        keyword_score = sum(2 for keyword in q["answer"].lower().split() if keyword in answer.lower())
        length_score = min(len(answer.split()) / 5, 4)  # Up to 4 points for length
        
        question_score = min(keyword_score + length_score, 10)  # Max 10 points per question
        scores.append(question_score)
        
        detailed_grade = {
            "question": q["question"],
            "user_answer": answer,
            "correct_answer": q["answer"],
            "keyword_score": keyword_score,
            "length_score": length_score,
            "total_score": question_score
        }
        detailed_grades.append(detailed_grade)
    
    return scores, detailed_grades

def display_results(scores, detailed_grades):
    print("\nDetailed Grading Results:")
    total_score = sum(scores)
    print(f"Total Score: {total_score:.2f}/{len(scores) * 10}")
    for i, grade in enumerate(detailed_grades, 1):
        print(f"\nQuestion {i}:")
        print(f"  Q: {grade['question']}")
        print(f"  Your Answer: {grade['user_answer']}")
        print(f"  Correct Answer: {grade['correct_answer']}")
        print(f"  Keyword Score: {grade['keyword_score']}/10")
        print(f"  Length Score: {grade['length_score']:.2f}/4")
        print(f"  Total Question Score: {grade['total_score']:.2f}/10")

def generate_report(pros, cons, scores, detailed_grades, filename='report.txt'):
    with open(filename, 'w') as file:
        file.write("Report\n")
        file.write("="*40 + "\n\n")
        
        file.write("Pros:\n")
        file.write("-" * 40 + "\n")
        for pro in pros:
            file.write(f"{pro}\n")
        file.write("\n")
        
        file.write("Cons:\n")
        file.write("-" * 40 + "\n")
        for con in cons:
            file.write(f"{con}\n")
        file.write("\n")
        
        file.write("Detailed Grading Results:\n")
        file.write("-" * 40 + "\n")
        total_score = sum(scores)
        file.write(f"Total Score: {total_score:.2f}/{len(scores) * 10}\n\n")
        
        for i, grade in enumerate(detailed_grades, 1):
            file.write(f"Question {i}:\n")
            file.write(f"  Q: {grade['question']}\n")
            file.write(f"  Your Answer: {grade['user_answer']}\n")
            file.write(f"  Correct Answer: {grade['correct_answer']}\n")
            file.write(f"  Keyword Score: {grade['keyword_score']}/10\n")
            file.write(f"  Length Score: {grade['length_score']:.2f}/4\n")
            file.write(f"  Total Question Score: {grade['total_score']:.2f}/10\n\n")
    
    print(f"\nReport generated: {filename}")

if __name__ == "__main__":
    text = capture_speech()
    if text:
        words, comma_separated_text = separate_words_with_commas(text)
        save_to_csv(words)
        pros, cons = classify_words(words)
        print("Pros:", pros)
        print("Cons:", cons)
        
        questions = generate_questions(pros)
        if questions:
            print("\nQuestions based on the pros:")
            scores, detailed_grades = ask_questions(questions)
            display_results(scores, detailed_grades)
            generate_report(pros, cons, scores, detailed_grades)
        else:
            print("\nNo questions generated from the pros list.")


Say something!
Recognized text: I am very good in Python and Java but I know Basics of C
Pros: ['I', 'am', 'very', 'good', 'in', 'Python', 'and', 'Java', 'but']
Cons: ['I', 'know', 'Basics', 'of', 'C']

Questions based on the pros:
1. What are the main advantages of Python?


Your answer:  it is a reptile


2. What are the main advantages of Java?


Your answer:  a good programming language.



Detailed Grading Results:
Total Score: 3.60/20

Question 1:
  Q: What are the main advantages of Python?
  Your Answer: it is a reptile
  Correct Answer: The main advantages of Python include its ease of use, versatility, and large community support. It's widely used in various domains and has extensive libraries and frameworks.
  Keyword Score: 0/10
  Length Score: 0.80/4
  Total Question Score: 0.80/10

Question 2:
  Q: What are the main advantages of Java?
  Your Answer: a good programming language.
  Correct Answer: The main advantages of Java include its ease of use, versatility, and large community support. It's widely used in various domains and has extensive libraries and frameworks.
  Keyword Score: 2/10
  Length Score: 0.80/4
  Total Question Score: 2.80/10

Report generated: report.txt
